In [26]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from scikeras.wrappers import KerasClassifier, KerasRegressor    # Regessor for regression tasks and Classifier for classification tasks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df= pd.read_csv("C:\\Users\\KUNAL\\.cache\\kagglehub\\datasets\\saurabhbadole\\bank-customer-churn-prediction-dataset\\versions\\2\\Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
gen_encoder = LabelEncoder()
df["Gender"] = gen_encoder.fit_transform(df["Gender"])
geo_encoder = OneHotEncoder(sparse_output=False)
geo_encoded_col = geo_encoder.fit_transform(df[["Geography"]])
geo_df = pd.DataFrame(geo_encoded_col, columns=geo_encoder.get_feature_names_out(["Geography"]))


In [5]:
df = pd.concat([df, geo_df], axis=1)
df = df.drop(columns=["Geography"], axis=1)

In [6]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
X = df.drop("EstimatedSalary", axis=1)
y = df["EstimatedSalary"]

In [8]:
X_train, X_test, y_train , y_test = train_test_split(X, y,  test_size=0.2, random_state=42)

In [9]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [30]:
# function to create a model and try differernt hyperparameters(KerasClassifier)
def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(neurons, input_dim=(X_train.shape[1]), ))

    for _ in range(layers -1):  # loop until the number of layers
        model.add(Dense(neurons, activation='relu'))

    model.add(Dense(1, activation="linear")) # output layer for classification
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

    return model

In [31]:
# kerasClassifier is a wrapper that allows Keras models to be used in scikit-learn
model = KerasRegressor(layers=1, neurons = 32,build_fn=create_model, epochs=50, batch_size=10, verbose=0)

In [32]:
# Define the grid search parameters
param = {
    "neurons": [16,32,64,128],
    "layers":[1,2,3],
    "batch_size":[10,20,30],
    "epochs":[50,100,150]
}

In [29]:
# Perform grid search

grid = GridSearchCV(estimator=model, param_grid = param, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters and score
print(f"Best Parameters: {grid_result.best_params_} and score: {grid_result.best_score_}")

c:\Users\KUNAL\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\KUNAL\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best Parameters: {'batch_size': 20, 'epochs': 150, 'layers': 2, 'neurons': 64} and score: -0.006576855038992043
